In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
import time

import tensorflow as tf
tf.random.set_seed(42)

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / max(y_true))) * 100

def RMSELoss(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
# Specifying the path of file
path= r"C:\Users\sashah8\OneDrive - North Carolina State University\Desktop\Q4\Data\L2\data_cleaned"

# Listing just the required files
fileList = os.listdir(path)
fileList.remove("Date.csv")

# Previewing the required file names
print(fileList)

['COAST.csv', 'EAST.csv', 'FWEST.csv', 'NCENT.csv', 'NORTH.csv', 'SCENT.csv', 'SOUTH.csv', 'WEST.csv']


In [ ]:
# Just for previewing the columns
pd.read_csv(os.path.join(path, fileList[7]))

,Net,Temprature
0,1470.814721,-7.1
1,1440.832944,-6.9
2,1430.495044,-6.5
3,1423.806348,-6.1
4,1428.180823,-5.7
...,...,...
43818,1290.258402,11.7
43819,1276.748646,12.2
43820,1260.087906,12.2
43821,1238.412254,11.6


### PROCESSING THE DATA

In [ ]:
# Restricting the load data to a single zone
fileName  = fileList[7]

zoneData = pd.read_csv(os.path.join(path, fileName))
zoneData.columns = ["Load_"+str(fileName[:-4]), "Temp_"+str(fileName[:-4])]

# For fixing one missing row
zoneData = pd.concat([zoneData.loc[[0]], zoneData], axis= 0).reset_index(drop= True)

zoneData

,Load_WEST,Temp_WEST
0,1470.814721,-7.1
1,1470.814721,-7.1
2,1440.832944,-6.9
3,1430.495044,-6.5
4,1423.806348,-6.1
...,...,...
43819,1290.258402,11.7
43820,1276.748646,12.2
43821,1260.087906,12.2
43822,1238.412254,11.6


### FEATURE ENGINEERING AND SPLITTING INTO TRAIN-TEST SPLIT

In [ ]:
def lagGenerator(dF, lagList):
    dF_temp = dF.filter(regex= "Load").copy()
    
    for i in lagList:
        dF_temp.loc[:, "Lag-" + str(i)] = dF_temp.iloc[:,[0]].shift(i*24)
    
    return(dF_temp.filter(regex= "Lag"))

In [ ]:
Load_lags = lagGenerator(dF=zoneData, lagList=[2,7])
Load_lags.head(2)

,Lag-2,Lag-7
0,NaN,NaN
1,NaN,NaN


In [ ]:
dataDf = pd.concat([zoneData, Load_lags], axis= 1).dropna(axis= 0).reset_index(drop= True)
dataDf

,Load_WEST,Temp_WEST,Lag-2,Lag-7
0,1500.085150,-3.3,1168.151890,1470.814721
1,1481.651866,-4.1,1145.834488,1470.814721
2,1481.418729,-4.5,1143.246135,1440.832944
3,1497.815892,-4.9,1151.230800,1430.495044
4,1518.714329,-5.7,1163.817932,1423.806348
...,...,...,...,...
43651,1290.258402,11.7,1312.208955,1153.226635
43652,1276.748646,12.2,1327.366356,1149.420623
43653,1260.087906,12.2,1337.492239,1139.002281
43654,1238.412254,11.6,1322.793472,1124.153791


In [ ]:
# Setting the train-test split ratio
ratio = 0.1

In [ ]:
Num_train, Num_test = int(len(dataDf)/24 * (1 - ratio)) * 24, int(len(dataDf)/24 * ratio) * 24

In [ ]:
# Splitting the data into train and test
data_train, data_test = dataDf[:Num_train], dataDf[Num_train:]

In [ ]:
Num_train, Num_valid = int(len(data_train)/24 * (1 - ratio)) * 24, int(len(data_train)/24 * ratio) * 24

In [ ]:
# Splitting the data into train and test
data_train, data_valid = data_train[:Num_train], data_train[Num_train:]

### SPLITTING INTO X AND Y PART WITH SEQUENCE BATCHING

In [ ]:
X_train_df, Y_train_df = data_train.iloc[:, 1:], data_train.iloc[:, :1]

X_valid_df, Y_valid_df = data_valid.iloc[:, 1:], data_valid.iloc[:, :1]

X_test_df, Y_test_df = data_test.iloc[:, 1:], data_test.iloc[:, :1]

In [ ]:
# Preparing the scaler
ValScaler = MinMaxScaler()

X_train = ValScaler.fit_transform(X_train_df)
X_valid = ValScaler.transform(X_valid_df)
X_test = ValScaler.transform(X_test_df)

Y_train = ValScaler.fit_transform(Y_train_df)
Y_valid = ValScaler.transform(Y_valid_df)
Y_test = ValScaler.transform(Y_test_df)

In [ ]:
def splitData(dataset):
    data = np.array(np.split(dataset, len(dataset)/24))
    return data

In [ ]:
def supervisedSet(X_part, Y_part, steps= 24):
    X_array, Y_array = np.array(X_part), np.array(Y_part)
    X, Y = [], []
    
    for i in range(len(X_array)):
        endIdx = i + steps
        
        if endIdx > len(X_array):
            break
        
        batch_X, batch_Y = X_array[i:endIdx], Y_array[i:endIdx]
        
        X.append(batch_X)
        Y.append(batch_Y)
        
    return np.array(X), np.array(Y)

In [ ]:
X_train, Y_train = supervisedSet(X_train, Y_train)

X_valid, Y_valid = splitData(X_valid), splitData(Y_valid)

X_test, Y_test = splitData(X_test), splitData(Y_test)

### WORKING ON THE MODEL

In [ ]:
epochs = 100
batchSize = 32

In [ ]:
# Encoder-decoder architecture [tanh required to use tensorflow on gpu]
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units= 128, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2]), dropout= 0.2),
    tf.keras.layers.RepeatVector(Y_train.shape[1]),
    tf.keras.layers.LSTM(units= 128, activation= "tanh", return_sequences= True, dropout= 0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 128, activation= "relu")),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 1))
])

model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [ ]:
# model = tf.keras.Sequential([
#     tf.keras.layers.LSTM(units= 256, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2])),
#     tf.keras.layers.Dense(units= 128, activation= "tanh"),
#     tf.keras.layers.Dense(units= 1)
# ])

# model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [ ]:
# Storing the start time of model fitting process
startTime = time.time()

In [ ]:
model.fit(X_train, Y_train, epochs= epochs, batch_size= batchSize, verbose= 1, shuffle= True, validation_data= (X_valid, Y_valid))

Epoch 1/100
1105/1105 [==============================] - 23s 17ms/step - loss: 0.0110 - mape: 2994.9900 - mae: 0.0786 - val_loss: 0.0086 - val_mape: 20.9761 - val_mae: 0.0740
Epoch 2/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0062 - mape: 2367.7983 - mae: 0.0600 - val_loss: 0.0178 - val_mape: 25.1984 - val_mae: 0.1002
Epoch 3/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0044 - mape: 2078.8293 - mae: 0.0510 - val_loss: 0.0231 - val_mape: 33.4428 - val_mae: 0.1166
Epoch 4/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0040 - mape: 1836.7682 - mae: 0.0483 - val_loss: 0.0114 - val_mape: 22.5847 - val_mae: 0.0835
Epoch 5/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0037 - mape: 1670.3058 - mae: 0.0464 - val_loss: 0.0194 - val_mape: 26.8834 - val_mae: 0.1029
Epoch 6/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0034 - mape: 1631.9789 - mae: 0.0447 - val_l

1105/1105 [==============================] - 19s 17ms/step - loss: 0.0014 - mape: 846.5731 - mae: 0.0290 - val_loss: 0.0235 - val_mape: 29.7893 - val_mae: 0.1154
Epoch 48/100
1105/1105 [==============================] - 19s 17ms/step - loss: 0.0014 - mape: 1071.1902 - mae: 0.0292 - val_loss: 0.0212 - val_mape: 26.3011 - val_mae: 0.1044
Epoch 49/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0014 - mape: 1384.2742 - mae: 0.0288 - val_loss: 0.0217 - val_mape: 27.1125 - val_mae: 0.1069
Epoch 50/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0014 - mape: 1367.7317 - mae: 0.0287 - val_loss: 0.0202 - val_mape: 27.2751 - val_mae: 0.1059
Epoch 51/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0014 - mape: 1105.6112 - mae: 0.0285 - val_loss: 0.0188 - val_mape: 25.2613 - val_mae: 0.0995
Epoch 52/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0013 - mape: 1159.6285 - mae: 0.0281 - val_loss: 0.0

1105/1105 [==============================] - 17s 15ms/step - loss: 8.4277e-04 - mape: 1050.6471 - mae: 0.0221 - val_loss: 0.0229 - val_mape: 26.3826 - val_mae: 0.1059
Epoch 94/100
1105/1105 [==============================] - 17s 16ms/step - loss: 8.3633e-04 - mape: 921.6705 - mae: 0.0221 - val_loss: 0.0188 - val_mape: 23.9770 - val_mae: 0.0967
Epoch 95/100
1105/1105 [==============================] - 17s 16ms/step - loss: 8.1987e-04 - mape: 1205.6938 - mae: 0.0218 - val_loss: 0.0222 - val_mape: 26.7516 - val_mae: 0.1065
Epoch 96/100
1105/1105 [==============================] - 17s 16ms/step - loss: 8.0554e-04 - mape: 843.7469 - mae: 0.0217 - val_loss: 0.0211 - val_mape: 26.7018 - val_mae: 0.1049
Epoch 97/100
1105/1105 [==============================] - 17s 15ms/step - loss: 8.2107e-04 - mape: 1143.3965 - mae: 0.0218 - val_loss: 0.0212 - val_mape: 26.4924 - val_mae: 0.1044
Epoch 98/100
1105/1105 [==============================] - 17s 15ms/step - loss: 8.0034e-04 - mape: 860.9532 - mae: 

In [ ]:
calcExecTime = (time.time() - startTime)
print('Total execution time in seconds: ' + str(calcExecTime))

Total execution time in seconds: 1779.2953646183014


### MAKING PREDICTIONS AND GENERATING RESULTS

In [ ]:
predictions = model.predict(X_test)

6/6 [==============================] - 1s 10ms/step


In [ ]:
predArr = predictions.reshape(predictions.shape[0] * predictions.shape[1], predictions.shape[2]).ravel()
predArr

array([0.19962908, 0.17727242, 0.16035996, ..., 0.45366153, 0.41105276,
       0.3795947 ], dtype=float32)

In [ ]:
Y_testArr = Y_test.reshape(Y_test.shape[0] * Y_test.shape[1], Y_test.shape[2]).ravel()
Y_testArr

array([0.3691414 , 0.34211376, 0.31706457, ..., 0.39183375, 0.37582581,
       0.36044968])

In [ ]:
def inverse_transform(df):
    column_names = df.columns
    for col in column_names:
        df[col] = ValScaler.inverse_transform(df[[col]]) 
    return df

In [ ]:
comparisionDf = pd.DataFrame(data= {"Values": Y_testArr, "Predictions": predArr})
comparisionDf = inverse_transform(comparisionDf)
comparisionDf

,Values,Predictions
0,1229.361185,999.831970
1,1192.764246,969.559875
2,1158.846247,946.659424
3,1142.629452,933.335510
4,1154.057787,936.352112
...,...,...
4363,1290.258402,1408.246704
4364,1276.748646,1392.676147
4365,1260.087906,1343.806274
4366,1238.412254,1286.111572


In [ ]:
mape(comparisionDf["Values"], comparisionDf["Predictions"])

11.235497806639652